# 5.6 Affine ／ Softmaxレイヤの実装

## 5.6.1 Affineレイヤ
ニューラルネットワークの順伝播で行う行列の積は、幾何学の分野では「アフィン変換」と呼ばれます。そのため、ここではアフィン変換を行う処理を「Affineレイヤ」という名前にします。

順伝播の場合は下記の計算式となります。
\begin{aligned}
\mathbf{Y} = \mathbf{X} \cdot \mathbf{W} + \mathbf{B}
\end{aligned}

逆伝播の場合は、下記の式が得られます。
\begin{aligned}
\frac{\partial L}{\partial \mathbf{X}} &= \frac{\partial L}{\partial \mathbf{Y}} \cdot \mathbf{W}^T \\[8px]
\frac{\partial L}{\partial \mathbf{W}} &= \mathbf{W}^T  \cdot \frac{\partial L}{\partial \mathbf{Y}}
\end{aligned}

## 5.6.2 バッチ版Affineレイヤ

In [1]:
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 重み・バイアスパラメータの微分
        self.dW = None
        self.db = None

    def forward(self, x):
        # テンソル対応
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 入力データの形状に戻す（テンソル対応）
        return dx

## 5.6.3 Softmax-with-Lossレイヤ

出力層であるソフトマックス関数について説明します。ソフトマックス関数は、（復習になりますが）入力された値を正規化して出力します。（出力の輪が１になるように変形） \
ここでは、損失関数である交差エントロピー誤差（cross entropy error）も含めて、「Softmax-with-Lossレイヤ」という名前のレイヤで実装します。

In [2]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None # softmaxの出力
        self.t = None # 教師データ

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 教師データがone-hot-vectorの場合
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx
